In [2]:
import numpy as np
import pandas as pd
import os
from sklearn import datasets

# Load Iris dataset, then join labels and features together
iris = datasets.load_iris()
joined_iris = np.insert(iris.data, 0, iris.target, axis=1)

df = pd.DataFrame(joined_iris, columns = ['label','sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)'])
train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))])

# Create a temporary directory and write the dataset as CSV
os.makedirs('./data', exist_ok=True)
#np.savetxt('./data/iris.csv', joined_iris, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')
train_data.to_csv('data/train.csv', index=False, header=False)
validation_data.to_csv('data/validation.csv', index=False, header=False)
test_data.to_csv('data/test.csv', index=False, header=False)

In [21]:
test_y = test_data.iloc[:,0]
test_X = test_data.iloc[:,1:]

In [ ]:
# Prerequisites:
#   1. Install required Python packages:
#      `pip install 'sagemaker[local]' 
#   2. Docker Desktop installed and running on your computer:
#      `docker ps`
#   3. You should have AWS credentials configured on your local machine
#      in order to be able to pull the docker image from ECR.

!pip install 'sagemaker[local]' --upgrade

%env AWS_ACCESS_KEY_ID=<Your Access Key>
%env AWS_SECRET_ACCESS_KEY=<Your Secret Key>
%env AWS_SESSION_TOKEN=<Your Session Token (if needed)>

#!aws configure set aws_access_key_id $AWS_ACCESS_KEY_ID
#!aws configure set aws_secret_access_key $AWS_SECRET_ACCESS_KEY
#!aws configure set aws_session_token $AWS_SESSION_TOKEN
#!aws configure set default.region $REGION

In [30]:
from sagemaker.local import LocalSession

sess_local = LocalSession()
sess_local.config = {'local': {'local_code': True} }

In [46]:
from sagemaker.sklearn.estimator import SKLearn

DUMMY_IAM_ROLE = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

sklearn = SKLearn('scikit_learn_iris.py', #/opt/ml/code/<script file>
                  instance_type='local',
                  instance_count=1,
                  framework_version='0.20.0',
                  py_version='py3',
                  role=DUMMY_IAM_ROLE,
                  sagemaker_session=sess_local,
                  metric_definitions=[{"Name": "model_accuracy", 
                                       "Regex": "Model Accuracy: ([0-9.]+).*$"}
                                     ],
                  hyperparameters={'max_leaf_nodes': 30} 
                  #/opt/ml/input/config/hyperparameters.json
                )

sklearn.fit({"train": 'file://./data/train.csv', "test": 'file://./data/validation.csv'})

Using the short-lived AWS credentials found in session. They might expire while running.


Login Succeeded
Creating network "sagemaker-local" with the default driver
Creating bvjcn8gjx6-algo-1-lorhl ... 
Creating bvjcn8gjx6-algo-1-lorhl ... done
Attaching to bvjcn8gjx6-algo-1-lorhl
bvjcn8gjx6-algo-1-lorhl | 2022-06-07 19:24:13,569 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
bvjcn8gjx6-algo-1-lorhl | 2022-06-07 19:24:13,573 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
bvjcn8gjx6-algo-1-lorhl | 2022-06-07 19:24:13,585 sagemaker_sklearn_container.training INFO     Invoking user training script.
bvjcn8gjx6-algo-1-lorhl | 2022-06-07 19:24:13,593 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
bvjcn8gjx6-algo-1-lorhl | 2022-06-07 19:24:13,612 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
bvjcn8gjx6-algo-1-lorhl | 2022-06-07 19:24:13,630 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
bvjcn8gjx6-al

In [47]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type='local')

print("Predictions: {}".format(predictor.predict(test_X.values)))
print("Actual: {}".format(test_y.values))

Using the short-lived AWS credentials found in session. They might expire while running.


Attaching to vvxtz9eufd-algo-1-vdxj8
vvxtz9eufd-algo-1-vdxj8 | Processing /opt/ml/code
vvxtz9eufd-algo-1-vdxj8 |   DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
vvxtz9eufd-algo-1-vdxj8 |    pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
vvxtz9eufd-algo-1-vdxj8 | Building wheels for collected packages: scikit-learn-iris
vvxtz9eufd-algo-1-vdxj8 |   Building wheel for scikit-learn-iris (setup.py) ... done
vvxtz9eufd-algo-1-vdxj8 |   Created wheel for scikit-learn-iris: filename=scikit_learn_iris-1.0.0-py2.py3-none-any.whl size=32124 sha256=74c1a5c394c2619244c2f4c6794df9193886e4ce09e42531ca97cb22b6949829
vvxtz9eufd-algo-1-vdxj8 |   Stored in directory: /tmp/pip-ephem-wheel-cache-y9rmbicj/wheels/3e